## Linux下交替读取两个文件内容方法总结

### 1.paste command

The paste command is the easiest of all to read 2 files simultaneously.  
paste prints the contents of multiple files side by side.  
By using the delimiter(-d) as the newline, the lines come to the new line.

In [1]:
head server.name server.ip

==> server.name <==
zzgame10
zzgame17
zzgame33
zzgame49
zzgame65
zzgame89

==> server.ip <==
175.25.18.171
175.25.18.137
175.25.18.138
175.25.18.144
175.25.18.143
175.25.18.139


In [2]:
paste -d'\n' server.name server.ip

zzgame10
175.25.18.171
zzgame17
175.25.18.137
zzgame33
175.25.18.138
zzgame49
175.25.18.144
zzgame65
175.25.18.143
zzgame89
175.25.18.139


In [3]:
# However,with this method,
# you can read n files in parallel
# such as read 3 files line by line:
head servername.txt server.ip server.name

==> servername.txt <==
黄巾起义
夷陵之战
巨马水之战
兴势之战
大鹏展翅
兵临城下

==> server.ip <==
175.25.18.171
175.25.18.137
175.25.18.138
175.25.18.144
175.25.18.143
175.25.18.139

==> server.name <==
zzgame10
zzgame17
zzgame33
zzgame49
zzgame65
zzgame89


In [4]:
paste -d'\n' servername.txt server.ip server.name

黄巾起义
175.25.18.171
zzgame10
夷陵之战
175.25.18.137
zzgame17
巨马水之战
175.25.18.138
zzgame33
兴势之战
175.25.18.144
zzgame49
大鹏展翅
175.25.18.143
zzgame65
兵临城下
175.25.18.139
zzgame89


### 2.paste with awk

awk uses newline as the output field separator(OFS)   
and hence the lines from both the files come in alternate lines.

In [7]:
head server.name server.ip

==> server.name <==
zzgame10
zzgame17
zzgame33
zzgame49
zzgame65
zzgame89

==> server.ip <==
175.25.18.171
175.25.18.137
175.25.18.138
175.25.18.144
175.25.18.143
175.25.18.139


In [8]:
# the paste default output
paste server.name server.ip

zzgame10	175.25.18.171
zzgame17	175.25.18.137
zzgame33	175.25.18.138
zzgame49	175.25.18.144
zzgame65	175.25.18.143
zzgame89	175.25.18.139


In [9]:
paste server.name server.ip | awk '$1=$1' OFS='\n'

zzgame10
175.25.18.171
zzgame17
175.25.18.137
zzgame33
175.25.18.138
zzgame49
175.25.18.144
zzgame65
175.25.18.143
zzgame89
175.25.18.139


### 3.awk option

The variable 'x' is assigned the 2nd filename "server.ip".   
The '1' command prints the lines of "server.name".   
getline command reads the contents of "server.ip" and prints one line at a time.   
In this fashion, the lines of both the files get printed in alternate fashion.  

In [5]:
head server.name server.ip

==> server.name <==
zzgame10
zzgame17
zzgame33
zzgame49
zzgame65
zzgame89

==> server.ip <==
175.25.18.171
175.25.18.137
175.25.18.138
175.25.18.144
175.25.18.143
175.25.18.139


In [6]:
awk '1;{getline < x;}1' x="server.ip" server.name

zzgame10
175.25.18.171
zzgame17
175.25.18.137
zzgame33
175.25.18.138
zzgame49
175.25.18.144
zzgame65
175.25.18.143
zzgame89
175.25.18.139


### 4.paste output is piped to the while loop

The output of paste command is piped to the while loop.  
It reads lines from "server.name" into f1 variable, "server.ip" to f2 variable, and prints them inside.   
Note: This way will work only if your file contains only one word.

In [12]:
head server.name server.ip

==> server.name <==
zzgame10
zzgame17
zzgame33
zzgame49
zzgame65
zzgame89

==> server.ip <==
175.25.18.171
175.25.18.137
175.25.18.138
175.25.18.144
175.25.18.143
175.25.18.139


In [15]:
paste server.name server.ip | 
while read f1 f2
do
    echo $f1
    echo $f2
done

zzgame10
175.25.18.171
zzgame17
175.25.18.137
zzgame33
175.25.18.138
zzgame49
175.25.18.144
zzgame65
175.25.18.143
zzgame89
175.25.18.139


### 5.Pure shell script

2 file descriptors 3 and 4 are assigned to "server.name" to "server.ip".  
Using the read command, the files are read one by one and they are printed.   
The loop is quit when there are no more lines to read in either files.   
The ':' in the while loop indicates a loop without any condition.   
This solution uses only internal commands.  

In [17]:
cat alternative.sh

#!/bin/bash
exec 3< $1
exec 4< $2
while :
do
    read x <&3
    read y <&4
    [ -n "$x" ] && echo "$x"
    [ -n "$y" ] && echo "$y"
    [ -z "$x" -a -z "$y" ] && break
done


In [18]:
./alternative.sh server.name server.ip

zzgame10
175.25.18.171
zzgame17
175.25.18.137
zzgame33
175.25.18.138
zzgame49
175.25.18.144
zzgame65
175.25.18.143
zzgame89
175.25.18.139
